In [3]:
# ! pip install pendulum

     |████████████████████████████████| 124 kB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 489 kB 2.0 MB/s eta 0:00:01


In [4]:
import pandas as pd
import numpy as np
import statsmodels
import matplotlib.pyplot as plt
import seaborn as sns
import pendulum

In [25]:
# reading revenue data 
data = pd.read_excel("UWCapstone_Revenue_4.0.xlsx", sheet_name= "AnonDataFinal")
data

,anon column,Industry,Month,First Month Flag,% change,First Month $ SML,LT % Growth Flags,Top 25% Dollars Generated
0,40065,C,7,0,1.000000,S,Super High,Top 25%
1,40065,C,8,1,8.165329,S,Super High,Top 25%
2,40065,C,9,1,-0.795248,S,Super High,Top 25%
3,40065,C,10,1,3.238761,S,Super High,Top 25%
4,40065,C,11,1,-0.462965,S,Super High,Top 25%
...,...,...,...,...,...,...,...,...
30068,50055,K,28,1,0.045604,XS,High,Bottom 25%
30069,50055,K,29,1,-0.013923,XS,High,Bottom 25%
30070,50055,K,30,1,2.232095,XS,High,Bottom 25%
30071,50055,K,31,1,17.748280,XS,High,Bottom 25%


In [29]:
data_grouped = data.groupby(["anon column","Industry","First Month $ SML","LT % Growth Flags",
                             "Top 25% Dollars Generated"],as_index = False).agg( frequency = ('First Month Flag','sum'),
                                                                sum_of_perc_change = ('% change', 'sum'),
                                                                max_of_month = ('Month','max'))
data_grouped

,anon column,Industry,First Month $ SML,LT % Growth Flags,Top 25% Dollars Generated,frequency,sum_of_perc_change,max_of_month
0,40065,C,S,Super High,Top 25%,26,186.114428,33
1,40075,J,XL,Super High,Top 25%,32,7.592392,33
2,40085,H,M,Super High,Top 25%,32,22.564491,33
3,40095,F,XL,High,Top 25%,31,2041.076124,33
4,40105,J,M,High,Top 25%,31,87.488798,33
...,...,...,...,...,...,...,...,...
994,50015,X,XS,Flat,Bottom 25%,32,127.227372,33
995,50025,CC,L,Medium,Bottom 25%,32,48.837718,33
996,50035,T,XS,Medium,Bottom 25%,32,836.173214,33
997,50045,D,XS,Medium,Bottom 25%,28,7447.664648,33


In [30]:
data_grouped.max_of_month.value_counts()

33    914
32     53
31     19
30      5
25      2
26      2
27      2
29      2
Name: max_of_month, dtype: int64

### Create Ranking 

In [31]:
revenue_data = data_grouped.copy()

In [32]:
revenue_data["First Month $ SML"].unique()

array(['S', 'XL', 'M', 'L', 'XS'], dtype=object)

In [33]:
revenue_data["LT % Growth Flags"].unique()

array(['Super High', 'High', 'Medium', 'Flat', 'Low'], dtype=object)

In [34]:
revenue_data["Top 25% Dollars Generated"].unique()

array(['Top 25%', 'Middle', 'Bottom 25%'], dtype=object)

In [35]:
# creating dummy variables for these categorical variables 

revenue_data["First_Month_SML_flag"] = np.where(revenue_data["First Month $ SML"] == 'XS',0,
                                               np.where(revenue_data["First Month $ SML"] == 'S',1,
                                                       np.where(revenue_data["First Month $ SML"] == 'M',2,
                                                               np.where(revenue_data["First Month $ SML"] == 'L',3,4))))

print(revenue_data["First_Month_SML_flag"].value_counts())
print(revenue_data["First Month $ SML"].value_counts())

0    354
1    308
2    146
4    104
3     87
Name: First_Month_SML_flag, dtype: int64
XS    354
S     308
M     146
XL    104
L      87
Name: First Month $ SML, dtype: int64


In [36]:
# creating dummy variables for these categorical variables 
# we consider flat as lowest as per the email 
revenue_data["LT_Growth_flag"] = np.where(revenue_data["LT % Growth Flags"] == 'Flat',0,
                                               np.where(revenue_data["LT % Growth Flags"] == 'Low',1,
                                                       np.where(revenue_data["LT % Growth Flags"] == 'Medium',2,
                                                               np.where(revenue_data["LT % Growth Flags"] == 'High',3,4))))

print(revenue_data["LT_Growth_flag"].value_counts())
print(revenue_data["LT % Growth Flags"].value_counts())

3    348
0    266
2    235
4     99
1     51
Name: LT_Growth_flag, dtype: int64
High          348
Flat          266
Medium        235
Super High     99
Low            51
Name: LT % Growth Flags, dtype: int64


In [37]:
# creating dummy variables for these categorical variables 

revenue_data["Top_25_Dollars_flag"] = np.where(revenue_data["Top 25% Dollars Generated"] == 'Top 25%',2,
                                               np.where(revenue_data["Top 25% Dollars Generated"] == 'Middle',1,0))

print(revenue_data["Top_25_Dollars_flag"].value_counts())
print(revenue_data["Top 25% Dollars Generated"].value_counts())

1    500
0    250
2    249
Name: Top_25_Dollars_flag, dtype: int64
Middle        500
Bottom 25%    250
Top 25%       249
Name: Top 25% Dollars Generated, dtype: int64


In [38]:
revenue_data.head()

,anon column,Industry,First Month $ SML,LT % Growth Flags,Top 25% Dollars Generated,frequency,sum_of_perc_change,max_of_month,First_Month_SML_flag,LT_Growth_flag,Top_25_Dollars_flag
0,40065,C,S,Super High,Top 25%,26,186.114428,33,1,4,2
1,40075,J,XL,Super High,Top 25%,32,7.592392,33,4,4,2
2,40085,H,M,Super High,Top 25%,32,22.564491,33,2,4,2
3,40095,F,XL,High,Top 25%,31,2041.076124,33,4,3,2
4,40105,J,M,High,Top 25%,31,87.488798,33,2,3,2


In [41]:
# revenue_data["Rank"] = revenue_data[["First_Month_SML_flag","LT_Growth_flag","Top_25_Dollars_flag"]].apply(tuple,axis=1)\
#              .rank(method='dense',ascending=True).astype(int)

"""
Creating rank by average instead regular method because we want to preserve the 
revenue %age change as part of account size 
"""


revenue_data["Rank"] = (revenue_data["First_Month_SML_flag"] + revenue_data["LT_Growth_flag"] + \
                        revenue_data["Top_25_Dollars_flag"]) / 3

revenue_data.sort_values(by = ["Rank"], ascending= False)

,anon column,Industry,First Month $ SML,LT % Growth Flags,Top 25% Dollars Generated,frequency,sum_of_perc_change,max_of_month,First_Month_SML_flag,LT_Growth_flag,Top_25_Dollars_flag,Rank
219,42255,H,XL,Super High,Top 25%,28,242.991540,32,4,4,2,3.333333
149,41555,H,XL,Super High,Top 25%,32,156.892487,33,4,4,2,3.333333
221,42275,AA,XL,Super High,Top 25%,32,140.610712,33,4,4,2,3.333333
25,40315,E,XL,Super High,Top 25%,32,27.620957,33,4,4,2,3.333333
1,40075,J,XL,Super High,Top 25%,32,7.592392,33,4,4,2,3.333333
...,...,...,...,...,...,...,...,...,...,...,...,...
761,47675,J,XS,Flat,Bottom 25%,32,173.432138,33,0,0,0,0.000000
782,47885,C,XS,Flat,Bottom 25%,32,223.063217,33,0,0,0,0.000000
831,48375,J,XS,Flat,Bottom 25%,31,218.120752,33,0,0,0,0.000000
765,47715,Q,XS,Flat,Bottom 25%,31,1665.825626,33,0,0,0,0.000000


In [42]:
# creatin new sales column by multiplying revenue growth with rank 

revenue_data["monetary"] = revenue_data["sum_of_perc_change"] * revenue_data["Rank"]

,anon column,Industry,First Month $ SML,LT % Growth Flags,Top 25% Dollars Generated,frequency,sum_of_perc_change,max_of_month,First_Month_SML_flag,LT_Growth_flag,Top_25_Dollars_flag,Rank,monetary
0,40065,C,S,Super High,Top 25%,26,186.114428,33,1,4,2,2.333333,434.267000
1,40075,J,XL,Super High,Top 25%,32,7.592392,33,4,4,2,3.333333,25.307974
2,40085,H,M,Super High,Top 25%,32,22.564491,33,2,4,2,2.666667,60.171976
3,40095,F,XL,High,Top 25%,31,2041.076124,33,4,3,2,3.000000,6123.228372
4,40105,J,M,High,Top 25%,31,87.488798,33,2,3,2,2.333333,204.140529


In [43]:
# creating recency column 

revenue_data["recency"] = 33 - revenue_data["max_of_month"]
revenue_data.head()

,anon column,Industry,First Month $ SML,LT % Growth Flags,Top 25% Dollars Generated,frequency,sum_of_perc_change,max_of_month,First_Month_SML_flag,LT_Growth_flag,Top_25_Dollars_flag,Rank,monetary,recency
0,40065,C,S,Super High,Top 25%,26,186.114428,33,1,4,2,2.333333,434.267000,0
1,40075,J,XL,Super High,Top 25%,32,7.592392,33,4,4,2,3.333333,25.307974,0
2,40085,H,M,Super High,Top 25%,32,22.564491,33,2,4,2,2.666667,60.171976,0
3,40095,F,XL,High,Top 25%,31,2041.076124,33,4,3,2,3.000000,6123.228372,0
4,40105,J,M,High,Top 25%,31,87.488798,33,2,3,2,2.333333,204.140529,0


In [47]:
revenue_data.recency.value_counts()

0    914
1     53
2     19
3      5
4      2
6      2
7      2
8      2
Name: recency, dtype: int64

In [55]:
rmf_data.describe()

,anon column,recency,frequency,monetary
count,999.000000,999.000000,999.000000,999.000000
mean,45055.710711,0.156156,29.096096,1361.507199
std,2886.452600,0.693254,5.760307,4252.717622
min,40065.000000,0.000000,4.000000,0.000000
25%,42560.000000,0.000000,30.000000,49.849177
50%,45055.000000,0.000000,32.000000,189.166690
75%,47550.000000,0.000000,32.000000,888.962397
max,50055.000000,8.000000,32.000000,63234.784603


In [56]:
#run RMF analysis using 3 buckets inactive, non-core, and core 
rmf_data = revenue_data[['anon column', 'recency', 'frequency', 'monetary']]
rmf_labels = ['inactive','non-core', 'core']
# reverse_labels = ['core', 'non-core', 'inactive']
# r_split = pd.qcut(rmf_data['recency'],
#                   [0, 0.2, 0.80, 1],
#                   duplicates='drop',
#                   labels=reverse_labels)
# rmf_data = rmf_data.assign(R = r_split.values)
# f_split = pd.qcut(rmf_data['frequency'], 
#                   [0, 0.2, 0.80, 1], 
#                   duplicates='drop',
#                   labels=rmf_labels)
# rmf_data = rmf_data.assign(F = f_split.values)
m_split = pd.qcut(rmf_data['monetary'], 
                  [0, 0.2, 0.80, 1], 
                  labels=rmf_labels)
rmf_data = rmf_data.assign(M = m_split.values)
print(rmf_data)

     anon column  recency  frequency     monetary         M
0          40065        0         26   434.267000  non-core
1          40075        0         32    25.307974  inactive
2          40085        0         32    60.171976  non-core
3          40095        0         31  6123.228372      core
4          40105        0         31   204.140529  non-core
..           ...      ...        ...          ...       ...
994        50015        0         32     0.000000  inactive
995        50025        0         32    81.396196  non-core
996        50035        0         32   557.448810  non-core
997        50045        0         28  4965.109766      core
998        50055        0         30    57.629538  non-core

[999 rows x 5 columns]


In [57]:
rmf_data.M.value_counts()

non-core    599
inactive    200
core        200
Name: M, dtype: int64

In [74]:
def score(x):
    if x['recency'] == 0 and x['frequency'] >= 30 and x['M'] == 'core':
        return 'Core'
    elif x['recency'] > 1 and x['frequency'] < 30 and x['M'] == 'inactive':
        return 'Inactive'
    else:
        return 'Non Core'
rmf_data['rfm_score'] = rmf_data.apply(score, axis=1)

In [75]:
core_customers = rmf_data[rmf_data['rfm_score'] == 'Core']
noncore_customers = rmf_data[rmf_data['rfm_score'] == 'Non Core']
inactive_customers = rmf_data[rmf_data['rfm_score'] == 'Inactive']
print('Number of Core Customers: ', len(core_customers))
print('Number of Non-Core Customers: ', len(noncore_customers))
print('Number of Inactive Customers: ', len(inactive_customers))

Number of Core Customers:  92
Number of Non-Core Customers:  905
Number of Inactive Customers:  2


In [76]:
rmf_data

,anon column,recency,frequency,monetary,M,rfm_score
0,40065,0,26,434.267000,non-core,Non Core
1,40075,0,32,25.307974,inactive,Non Core
2,40085,0,32,60.171976,non-core,Non Core
3,40095,0,31,6123.228372,core,Core
4,40105,0,31,204.140529,non-core,Non Core
...,...,...,...,...,...,...
994,50015,0,32,0.000000,inactive,Non Core
995,50025,0,32,81.396196,non-core,Non Core
996,50035,0,32,557.448810,non-core,Non Core
997,50045,0,28,4965.109766,core,Non Core


In [77]:
rmf_data.M.value_counts()

non-core    599
inactive    200
core        200
Name: M, dtype: int64

In [70]:
def score(x):
    if x['recency'] == 0 and x['frequency'] >= 28 and x['M'] == 'core':
        return 'Core'
    elif x['recency'] > 1 and x['frequency'] < 28 and x['M'] == 'inactive':
        return 'Inactive'
    else:
        return 'Non Core'
rmf_data['rfm_score'] = rmf_data.apply(score, axis=1)

In [71]:
core_customers = rmf_data[rmf_data['rfm_score'] == 'Core']
noncore_customers = rmf_data[rmf_data['rfm_score'] == 'Non Core']
inactive_customers = rmf_data[rmf_data['rfm_score'] == 'Inactive']
print('Number of Core Customers: ', len(core_customers))
print('Number of Non-Core Customers: ', len(noncore_customers))
print('Number of Inactive Customers: ', len(inactive_customers))

Number of Core Customers:  108
Number of Non-Core Customers:  889
Number of Inactive Customers:  2


In [78]:
rmf_data

,anon column,recency,frequency,monetary,M,rfm_score
0,40065,0,26,434.267000,non-core,Non Core
1,40075,0,32,25.307974,inactive,Non Core
2,40085,0,32,60.171976,non-core,Non Core
3,40095,0,31,6123.228372,core,Core
4,40105,0,31,204.140529,non-core,Non Core
...,...,...,...,...,...,...
994,50015,0,32,0.000000,inactive,Non Core
995,50025,0,32,81.396196,non-core,Non Core
996,50035,0,32,557.448810,non-core,Non Core
997,50045,0,28,4965.109766,core,Non Core


In [79]:
rmf_data.to_csv("RFM.csv", index = False)